In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score


In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3aa3c4da03ff4732a81e56a3b678da70 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='grLB7-TuYRnbrDecUugCTYOnpq_rLThfAshhTOKIbYdO',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_3aa3c4da03ff4732a81e56a3b678da70.get_object(Bucket='lifeexpectancymodel-donotdelete-pr-2eeplhknxjt33d',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.

life_exp_data= pd.read_csv(body)
life_exp_data.head()

In [ ]:
life_exp_data=life_exp_data.drop('Year',axis=1)
life_exp_data=life_exp_data.drop(['Country'],axis=1)

In [ ]:
status_types=["Developing","Developed"]
label_churn=pd.DataFrame(life_exp_data,columns=["Status"]);
label_encoder=LabelEncoder();
label=life_exp_data["Status"]
life_exp_data["Status"]=label_encoder.fit_transform(label);


In [ ]:
life_exp_data.head().shape

In [ ]:
categorical_Columns=life_exp_data.select_dtypes(include=[np.object]).columns
print(categorical_Columns)

imput_categorical=SimpleImputer(strategy='mean');

onehot_categorical=OneHotEncoder(handle_unknown='ignore');

categorical_transformer=Pipeline(steps=[('impute',imput_categorical),('onehot',onehot_categorical)]);


In [ ]:

numerical_Columns=[col for col in life_exp_data.select_dtypes(include=[np.double,np.float,np.int]).columns if col not in ['Life_expectancy ']]
print(numerical_Columns)

num_scaler=StandardScaler()

numerical_transformer=Pipeline(steps=[('scaler',num_scaler)])

In [ ]:
preprocessorForCategoricalColumns= ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_Columns)],
                                            remainder="passthrough")
preprocessorForAllColumns = ColumnTransformer(transformers=[('cat', categorical_transformer, categorical_Columns),('num',numerical_transformer,numerical_Columns)],
                                            remainder="passthrough")
pre_temp=preprocessorForCategoricalColumns.fit_transform(life_exp_data)
print(pre_temp)
pre_temp1=preprocessorForAllColumns.fit_transform(life_exp_data)
print(pre_temp1)



In [ ]:
features = []
features = life_exp_data.drop(['Life_expectancy '], axis=1)

label = pd.DataFrame(life_exp_data, columns = ['Life_expectancy ']) 
label = life_exp_data['Life_expectancy ']


In [ ]:
corr_matrix = life_exp_data.corr()
corr_matrix['Life_expectancy '].sort_values(ascending=False)


In [ ]:
attributes=['Life_expectancy ','Schooling','Income_composition_of_resources','Adult_Mortality']
scatter_matrix(life_exp_data[attributes],figsize=(12,8));

In [ ]:
features.fillna(value = features.mean(), inplace = True)
label.fillna(value = label.mean(), inplace = True)

In [ ]:

X_train,X_test,y_train,y_test=train_test_split(features,label,test_size=0.2,random_state=42);

In [ ]:
print(f"records in the test set are :{len(X_train)} \n records in the test set are :{len(X_test)}")
print(X_train.head()) 

In [ ]:
#life_exp_data['Status'].value_counts()

X_test.fillna(X_test.mean())

X_train_slr, X_test_slr, y_train_slr, y_test_slr = train_test_split(features,label , random_state=0)

print("Dimensions of datasets that will be used for training : Input features"+str(X_train_slr.shape)+ 
      " Output label" + str(y_train_slr.shape))

print("Dimensions of datasets that will be used for testing : Input features"+str(X_test_slr.shape)+ 
      " Output label" + str(y_test_slr.shape))

life_exp_data.describe()



In [ ]:
def model_metrics(regressor,y_test,y_pred):
    mse = mean_squared_error(y_test,y_pred)
    print("Mean squared error: %.2f"
      % mse)
    r2 = r2_score(y_test, y_pred)
    print('R2 score: %.2f' % r2 )
    return [mse, r2]


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

model_name = 'Random Forest Regression'

slRegressor = RandomForestRegressor()

slRegressor.fit(X_train_slr,y_train_slr)

y_pred_slr= slRegressor.predict(X_test_slr)

print(slRegressor)

In [ ]:
predictions = slRegressor.predict(X_test)
life_exp_data.head()

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
print(f"Score :{slRegressor.score(X_test,y_test)}")
slrMetrics = model_metrics(slRegressor,y_test_slr,y_pred_slr)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#!pip install watson-machine-learning-client


In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_credentials={
  "apikey": "XQZfJcTXchdfJHeyCKGXpl28D7OWaZBVOg1WMfrMokYj",
  "apikey_name": "Service credentials-1",
  "instance_id": "5344ac7d-15a6-4e4c-a3ed-ac28bc2e84ac",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [ ]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Sandy", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "sandysawarkar20@gmail.com", 
               client.repository.ModelMetaNames.NAME: "Life_expectancy"}

In [ ]:
model_artifact =client.repository.store_model(slRegressor, meta_props=model_props)

In [ ]:
published_model_uid = client.repository.get_model_uid(model_artifact)

In [ ]:
published_model_uid

In [ ]:
deployment = client.deployments.create(published_model_uid, name="Life_expectancy")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(deployment)

In [ ]:
scoring_endpoint